# Rewrite Training Data

To allow for human observation of the training data, we rewrite the freebase mids to be human readable names.

In [1]:
import sys
sys.path.insert(0, '../../')
from tqdm import tqdm_notebook
tqdm_notebook().pandas()
from notebooksutils.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [8]:
from notebooksutils.simple_qa import load_simple_qa 

# Load development set because its a magnitude smaller than the training set.
df, = load_simple_qa(train=True)
df[:5]

,subject,relation,object,question
0,04whkz5,book/written_work/subjects,01cj3p,what is the book e about
1,0tp2p24,music/release_track/release,0sjc7c1,to what release does the release track cardiac...
2,04j0t75,film/film/country,07ssc,what country was the film the debt from
3,0ftqr,music/producer/tracks_produced,0p600l,what songs have nobuo uematsu produced?
4,036p007,music/release/producers,0677ng,Who produced eve-olution?


In [11]:
def get_aliases(mid):
    cursor.execute("SELECT alias FROM fb_two_name WHERE mid = %s", (mid, ))
    aliases = [r[0] for r in cursor.fetchall()]
    aliases = sorted(aliases, key=lambda a: len(a), reverse=True)
    return aliases[:3]

def update_row(row):
    subject_aliases = get_aliases(row['subject'])
    row['subject_aliases'] = subject_aliases
    subject_aliases = get_aliases(row['object'])
    row['object_aliases'] = subject_aliases
    return row

df = df.progress_apply(update_row, axis=1)
df[:5]

HBox(children=(IntProgress(value=0, max=75910), HTML(value='')))

,subject,relation,object,question,subject_aliases,object_aliases
0,04whkz5,book/written_work/subjects,01cj3p,what is the book e about,[e],[spiritualism]
1,0tp2p24,music/release_track/release,0sjc7c1,to what release does the release track cardiac...,[cardiac arrest],[complete madness]
2,04j0t75,film/film/country,07ssc,what country was the film the debt from,[the debt],[the united kingdom of great britain and north...
3,0ftqr,music/producer/tracks_produced,0p600l,what songs have nobuo uematsu produced?,"[uematsu, nobuo, nabuo uematsu, nobou uematsu]","[the oath (arranged version) (ffviii), the oat..."
4,036p007,music/release/producers,0677ng,Who produced eve-olution?,[eve-olution],"[kasseem david dean, kasseem mike dean, kaseem..."


In [14]:
DEST = '../../data/SimpleQuestions_v2/interpretable_train.tsv'
lines = []
for index, row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
    formatted_line = [row['question'], str(row['subject_aliases']), row['relation'], str(row['object_aliases'])]
    formatted_line = '\t'.join(formatted_line)
    lines.append(formatted_line)
file_ = open(DEST, 'w')
file_.write('\n'.join(lines))

HBox(children=(IntProgress(value=0, max=75910), HTML(value='')))

11154284